In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

In [2]:
conda env list

# conda environments:
#
base                     /home/huytran/miniconda3
FM                    *  /home/huytran/miniconda3/envs/FM
P2                       /home/huytran/miniconda3/envs/P2
TF                       /home/huytran/miniconda3/envs/TF


Note: you may need to restart the kernel to use updated packages.


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
df = pd.read_csv('./data/data_cleaned_3.csv', index_col=0)

In [5]:
df.head()

,id,host_id,host_since,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,city,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,email,facebook,google,government_id,identity_manual,jumio,kba,manual_offline,manual_online,offline_government_id,phone,photographer,reviews,selfie,sent_id,sesame,sesame_offline,weibo,work_email,zhima_selfie,24-hourcheck-in,Accessible-heightbed,Accessible-heighttoilet,Airconditioning,Airpurifier,Alfrescobathtub,AmazonEcho,AppleTV,BBQgrill,Babybath,Babymonitor,Babysitterrecommendations,Balcony,Bathroomessentials,Bathtowel,Bathtub,Bathtubwithbathchair,Beachessentials,Beachfront,Beachview,Bedlinens,Bedroomcomforts,Bidet,Bodysoap,Breakfast,Breakfastbar,Breakfasttable,Buildingstaff,Buzzer/wirelessintercom,CableTV,Carbonmonoxidedetector,Cat(s),Ceilingfan,Ceilingfans,Ceilinghoist,Centralairconditioning,Changingtable,Charcoalbarbeque,Chef'skitchen,Children’sbooksandtoys,Children’sdinnerware,Cleaningbeforecheckout,Coffeemaker,Convectionoven,Cookingbasics,Crib,DVDplayer,Daybed,Diningarea,Diningtable,Disabledparkingspot,Dishesandsilverware,Dishwasher,Dog(s),Doorman,Doubleoven,Dryer,Dualvanity,EVcharger,Electricprofilingbed,Elevator,Ensuitebathroom,Espressomachine,Essentials,Ethernetconnection,Exerciseequipment,Extrapillowsandblankets,Extraspacearoundbed,Family/kidfriendly,Faxmachine,Fireextinguisher,Firepit,Fireplaceguards,Firmmattress,Firstaidkit,Fixedgrabbarsforshower,Fixedgrabbarsfortoilet,Flatpathtoguestentrance,Formaldiningarea,Freeparkingonpremises,Freestreetparking,Fullkitchen,Gameconsole,Garden,Gardenorbackyard,Gasfireplace,Gasoven,Groundflooraccess,Gym,HBOGO,Hairdryer,Hammock,Handheldshowerhead,Hangers,Heatedfloors,Heatedtowelrack,Heating,Heatlamps,High-resolutioncomputermonitor,Highchair,Hostgreetsyou,Hottub,Hotwater,Hotwaterkettle,IceMachine,Indoorfireplace,Internet,Iron,IroningBoard,Jettedtub,Keypad,Kitchen,Kitchenette,Lakeaccess,Laptopfriendlyworkspace,Lockbox,Lockonbedroomdoor,Longtermstaysallowed,Luggagedropoffallowed,Mediaroom,Memoryfoammattress,Microwave,Minifridge,Mobilehoist,Mountainview,Mudroom,Murphybed,Naturalgasbarbeque,Nespressomachine,Netflix,Nostairsorstepstoenter,Office,Other,Otherpet(s),Outdoorkitchen,Outdoorparking,Outdoorseating,Outletcovers,Oven,Pack’nPlay/travelcrib,Paidparkingoffpremises,Paidparkingonpremises,Parking,Patioorbalcony,Petsallowed,Petsliveonthisproperty,Piano,Pillow-topmattress,Pocketwifi,Pool,Poolcover,Poolwithpoolhoist,Printer,Privatebathroom,Privateentrance,Privategym,Privatehottub,Privatelivingroom,Privatepool,Projectorandscreen,Propanebarbeque,Rainshower,Refrigerator,Rooftop,Room-darkeningshades,Safetycard,SatelliteTV,Sauna,Securitysystem,Selfcheck-in,Shampoo,Sharedgym,Sharedhottub,Sharedpool,Showerbathtubcombo,Showerchair,Singlelevelhome,Ski-in/Ski-out,SmartTV,Smartlock,Smokedetector,Smokingallowed,Soakingtub,Sonossoundsystem,Soundsystem,Stairgates,Standalonebathtub,Standalonerainshower,Standalonesteamshower,Standingvalet,Steamoven,Step-freeshower,Stove,Suitableforevents,Sunloungers,Surroundsoundsystem,TV,Tablecornerguards,Tenniscourt,Terrace,Toiletpaper,Touchlessfaucets,VideoGames,Walk-inshower,Warmingdrawer,Washer,Washer/Dryer,Waterfront,Well-litpathtoentrance,Wetbar,Wheelchairaccessible,Wideclearanceto

In [6]:
df.shape

(84489, 354)

In [7]:
df.columns

Index(['id', 'host_id', 'host_since', 'host_response_rate',
       'host_is_superhost', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified', 'city', 'latitude',
       ...
       'moderate', 'strict', 'strict_14_with_grace_period', 'super_strict_30',
       'super_strict_60', '0', 'a few days or more', 'within a day',
       'within a few hours', 'within an hour'],
      dtype='object', length=354)

In [8]:
df = pd.DataFrame.drop(df, columns=['id', 'host_id'])

In [9]:
numerics = ['int16','int32','int64','float16','float32','float64']
numerical_vars = list(df.select_dtypes(include=numerics).columns)
df = df[numerical_vars]
df.shape

(84489, 351)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df, df['price'], test_size=0.3, random_state=0)
X_train.shape, X_test.shape

((59142, 351), (25347, 351))

# **Lasso**

In [11]:
scaler = StandardScaler()
scaler.fit(X_train.fillna(0))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [ ]:
ALPHAS = [0.00016, 0.00018, 0.00020, 0.00022, 0.00024, 0.00026, 0.00028, 0.00030, 0.00032]

score_best = 0
for alpha in ALPHAS:
    print('alpha:', alpha)
    sel_ = Lasso(alpha=0.0001, max_iter=1e5)
    sel_.fit(X_train, y_train)
    score = sel_.score(X_test, y_test)
    print('\t training score:', sel_.score(X_train, y_train))
    print('\t validation score:', sel_.score(X_test, y_test))
    if score > score_best:
        score_best = score
        alpha_best = alpha

alpha: 0.00016


/home/huytran/miniconda3/envs/FM/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.432313487696336, tolerance: 3.625284836457473
  positive)


	 training score: 0.9999999621658111
	 validation score: 0.9999999619404283
alpha: 0.00018


In [1]:
# # make a list with the selected features and print the outputs
# selected_feat = X_train.columns[(sel_.get_support())]

# print('total features: {}'.format((X_train.shape[1])))
# print('selected features: {}'.format(len(selected_feat)))
# print('features with coefficients shrank to zero: {}'.format(
#     np.sum(sel_.estimator_.coef_ == 0)))

In [15]:
# X_train_selected = sel_.transform(X_train.fillna(0))
# X_test_selected = sel_.transform(X_test.fillna(0))
# X_train_selected.shape, X_test_selected.shape

((59154, 1), (25353, 1))

In [ ]:
print('best alpha:', alpha_best)

In [28]:
sel_ = Lasso(alpha=0.00000001, max_iter=1e3)
sel_.fit(X_train, y_train)
print('training set:', sel_.score(X_train, y_train))
print('validation set:', sel_.score(X_test, y_test))
# y_pred_train = sel_.predict(X_train)
# y_pred_val = sel_.predict(X_test)

print('total number of parameters:', sum(sel_.coef_!=0))

training set: 0.9999998325099917
validation set: 0.9999998253621039
total number of parameters: 246


In [29]:
coefs = np.array(sel_.coef_!=0)
np.save('./data/selected_coefs_4_0_00000001.npy', coefs)
print('total number of parameters:', sum(sel_.coef_!=0))

total number of parameters: 246


# **Random Forest**

In [5]:
data = pd.read_csv('./data/data_cleaned_3.csv')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, data['price'], test_size=0.3, random_state=0)

In [11]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)

ValueError: Unknown label type: 'continuous'

In [8]:
sel.get_support()

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True, False, False,
       False,  True, False, False,  True,  True, False, False, False,
       False,  True, False,  True, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False,  True, False, False, False,
        True, False, False, False,  True,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False,  True, False, False, False, False, False, False,
        True,  True, False, False, False,  True, False, False, False,
        True, False,

In [9]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

125

In [10]:
print(selected_feat)

Index(['Unnamed: 0', 'id', 'host_id', 'host_since', 'host_response_rate',
       'host_is_superhost', 'host_total_listings_count',
       'host_identity_verified', 'latitude', 'longitude',
       ...
       'Private room', 'greater-london', 'london', 'flexible', 'moderate',
       'strict_14_with_grace_period', '0', 'within a day',
       'within a few hours', 'within an hour'],
      dtype='object', length=125)


In [12]:
data_selected = data[selected_feat]

In [14]:
data_selected.shape

(84507, 125)

In [15]:
data_selected.to_csv('data_selected.csv')